In [2]:
import os
from tqdm import tqdm
import xml.etree.ElementTree as ET
from collections import defaultdict

In [3]:
word_align = ET.parse('./xces/en-vi.xml')

In [4]:
word_align_root = word_align.getroot()

In [5]:
word_align_root[10].attrib

{'fromDoc': 'en/1944/36644/5596733.xml.gz',
 'targType': 's',
 'toDoc': 'vi/1944/36644/5203600.xml.gz'}

In [6]:
targets = ['fromDoc', 'toDoc']

def load_words(doc_path):
    res = defaultdict(list)
    with open(os.path.join('./xces', doc_path[:-3]), 'rb') as f:
        from_doc = ET.fromstring(f.read())
        for sentence in from_doc.findall('s'):
            res[sentence.attrib['id']] = [word.text.lower() for word in sentence.findall('w')]
    return res

def is_in_sentence(word, sentence, l_word):
    for i in range(len(sentence)):
        if sentence[i:i+l_word] == word:
            return True
    return False

def find_sentence_with_word_in_doc(target_word, sentences):
    if sentences is None or len(sentences) <= 0:
        return None
    
    res = set([])
    l_target_word = len(target_word)
    for id, sentence in sentences.items():
        if is_in_sentence(target_word, sentence, l_target_word):
            res.add(id)

    return res

def find_word(target_word, target_doc):
    target_word = target_word.lower().split()
    
    res = []
    
    for film in tqdm(word_align_root):
        src_doc_sentences = load_words(film.attrib[targets[target_doc]])
        sentence_indexes = find_sentence_with_word_in_doc(target_word, src_doc_sentences)
        if sentence_indexes is None or len(sentence_indexes) <= 0:
            continue

        dst_doc_sentences = None
        for pair in film:
            dw_id = pair.attrib['xtargets'].split(';')
            if len(dw_id) != 2 or dw_id[0] == '' or dw_id[1] == '':
                continue
            found = False
            for i in dw_id[target_doc].split():
                if i in sentence_indexes:
                    found = True
                    break
            if found:
                if dst_doc_sentences is None:
                    dst_doc_sentences = load_words(film.attrib[targets[1-target_doc]])

                src_sen = []
                dst_sen = []
                for i in dw_id[target_doc].split():
                    src_sen += src_doc_sentences[i]
                for i in dw_id[1-target_doc].split():
                    dst_sen += dst_doc_sentences[i]
                res += [(src_sen, dst_sen)]
    
    return res

In [7]:
t = find_word('collect', 0)

100%|██████████| 2354/2354 [01:58<00:00, 19.80it/s]


In [8]:
len(t)

307

In [9]:
for (sen1, sen2) in t:
    print('----------------')
    print(' '.join(sen1))
    print(' '.join(sen2))

----------------
i can show bryan , our da , that if he goes around tryin ' to collect everybody he 'll have a tangled case but if he sticks to wilmer here , he can get a conviction standing on his head .
tôi có thể chỉ cho bryan , tay công tố viên , rằng nếu ổng cứ lo đi dò xét tất cả mọi người , thì ổng sẽ sa vào một mớ bòng bong , nhưng nếu ổng chú ý tới wilmer đây , thì ổng sẽ có thể dễ dàng buộc tội hắn .
----------------
would you send it to my hotel , collect ?
cô gởi tới khách sạn của tôi và nhận tiền ở đó được không ?
----------------
$ 20,000 against a six-foot hole in boot hill or a $ 20-a-month pension , if you live long enough to collect it .
20.000 đô thay vì một cái lỗ hai mét ở boot hill , hay lương hưu 20 đô mỗi tháng nếu anh sống đủ lâu để nhận được .
----------------
collect your tiny brain and try and do better this time .
cố gắng động cái não tí xíu của ông và lần này hãy làm tốt hơn .
----------------
she wouldn 't know how to collect it .
bả sẽ không biết cách lã

### Failed code

In [ ]:
from_doc_words = defaultdict(set)
to_doc_words = defaultdict(set)

for film in tqdm(word_align_root):
    with gzip.open(os.path.join('./xces', film.attrib['fromDoc']), 'rb') as f:
        from_doc = ET.fromstring(f.read())
        for word in from_doc.findall('./s/w'):
            from_doc_words[word].add(film.attrib['fromDoc'])
    with gzip.open(os.path.join('./xces', film.attrib['toDoc']), 'rb') as f:
        to_doc = ET.fromstring(f.read())
        for word in to_doc.findall('./s/w'):
            to_doc_words[word].add(film.attrib['toDoc'])